Let's see what we can do with enforcement actions

In [20]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from icecream import ic

In [21]:
enforcement_file = '..\enforcements\enforcements.csv'
enforces = pd.read_csv(enforcement_file)
enforces.head()

,Penalty Year,Agency Short,Action Type,Agency,Level of Government,Penalty Amount Adjusted For Eliminating Multiple Counting,Parent Short,Case ID,Civil/Criminal,Company,...,Parent at the Time of the Penalty Announcement,Penalty Amount,Penalty Amount Adjusted For Eliminating Multiple Counting (Primary Offense),Penalty Date,Primary Offense,Private Litigation Case Title,Prosecution Agreement,Recap of Ownership Changes,Secondary Offense,Subtraction From Penalty Amount to Avoid Double Counting
0,2000,Other Federal,agency action,Equal Employment Opportunity Commission,federal,1000000,Macquarie,NaN,civil,Cincinnati Bell Telephone and Cincinnati Bell ...,...,Cincinnati Bell,1000000,758892309,6/15/2000,employment discrimination,NaN,NaN,Macquarie bought Cincinnati Bell in 2021,pregnancy discrimination,0
1,2000,SEC/FINRA,agency action,Securities and Exchange Commission,federal,12941191,RBC,NaN,civil,Dain Rauscher Incorporated,...,Dain Rauscher Incorporated,12941191,80991516314,4/6/2000,investor protection violation,NaN,NaN,Royal Bank of Canada acquired Dain Rauscher in...,securities issuance or trading violation,0
2,2000,Other Federal,agency action,Office of Federal Contract Compliance Programs,federal,440000,5th/3rd,NaN,civil,Fifth Third Bank,...,Fifth Third Bancorp,440000,758892309,3/8/2000,employment discrimination,NaN,NaN,NaN,racial discrimination,0
3,2000,Other Federal,agency action,National Labor Relations Board,federal,9116,Wells,04-CA-27834-001,civil,First Union Bank,...,First Union,9116,1234941,1/5/2000,labor relations violation,NaN,NaN,"In 2001 First Union merged with Wachovia, whic...",NaN,0
4,2000,SEC/FINRA,agency action,Securities and Exchange Commission,federal,5200000,GS,NaN,civil,"Goldman, Sachs & Co.",...,Goldman Sachs,5200000,80991516314,4/6/2000,investor protection violation,NaN,NaN,NaN,securities issuance or trading violation,0


In [11]:
enforces.columns

Index(['Penalty Year', 'Agency Short', 'Action Type', 'Agency',
       'Level of Government',
       'Penalty Amount Adjusted For Eliminating Multiple Counting',
       'Parent Short', 'Case ID', 'Civil/Criminal', 'Company', 'Court',
       'Current Parent Company', 'Current Parent HQ Country',
       'Current Parent HQ State', 'Current Parent ISIN',
       'Current Parent Major Industry', 'Current Parent Ownership Structure',
       'Current Parent Specific Industry', 'Current Parent Stock Ticker',
       'Description', 'Facility Address', 'Facility City', 'Facility County',
       'Facility Name', 'Facility State', 'Facility Zip', 'Info Source',
       'Lawsuit Resolution', 'NAICS Code', 'NAICS Translation', 'Notes',
       'Offense Group', 'Original System ID', 'PACER Link',
       'Parent at the Time of the Penalty Announcement', 'Penalty Amount',
       'Penalty Amount Adjusted For Eliminating Multiple Counting (Primary Offense)',
       'Penalty Date', 'Primary Offense', 'Private

In [17]:
# Give me the columns names of the dataframe
query_columns = ['Penalty Year', 'Agency Short', 'Agency', 'Company', 'Current Parent Stock Ticker',
                 'Description', 'Facility County', 'Penalty Date', 'Info Source']
query = pd.DataFrame(enforces[query_columns])
# combine query columns into one string
query['query'] = query.astype(str).add(' ').sum(axis=1).str[:-1]
query['query'][0]

'2000 Other Federal Equal Employment Opportunity Commission Cincinnati Bell Telephone and Cincinnati Bell Information Systems MIC nan nan 6/15/2000 https://www.eeoc.gov/eeoc/newsroom/release/archive/6-15-00-a.html'

In [22]:
search_phrase = 'fine'

for q in query['query'][:2]:    
    print(q)
    search_url = f"https://www.google.com/search?q={query}"
    response = requests.get(search_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the search results page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find and iterate through search result links
        for link in soup.find_all('a'):
            href = link.get('href')
            ic(href)
            if href.startswith("/url?q="):
                url = href.replace("/url?q=", "").split('&')[0]              
                doc_response = requests.get(url)
                ic(url)
                if doc_response.status_code == 200:
                    # Parse the document content (you may need to adapt this for different document types)
                    doc_soup = BeautifulSoup(doc_response.text, 'html.parser')
                    doc_text = doc_soup.get_text()
                    
                    # Check if the phrase you're looking for is in the document
                    phrase_to_search = "your_phrase_here"
                    if phrase_to_search in doc_text:
                        # Save the document to a file
                        filename = f"{query}_{os.path.basename(url)}.txt"
                        # with open(filename, 'w', encoding='utf-8') as file:
                        #     file.write(doc_text)
                        print(f"Document saved: {filename}")

    else:
        print("Failed to retrieve search results")

2000 Other Federal Equal Employment Opportunity Commission Cincinnati Bell Telephone and Cincinnati Bell Information Systems MIC nan nan 6/15/2000 https://www.eeoc.gov/eeoc/newsroom/release/archive/6-15-00-a.html


ic| href: '/?sa=X&ved=0ahUKEwj894GTlvuCAxXgv4kEHed7DlQQOwgC'
ic| href: '/search?q=Penalty+Year+++Agency+Short++%5C%0A0+++++++++++++2000++Other+Federal+++%0A1+++++++++++++2000++++++SEC/FINRA+++%0A2+++++++++++++2000++Other+Federal+++%0A3+++++++++++++2000++Other+Federal+++%0A4+++++++++++++2000++++++SEC/FINRA+++%0A...++++++++++++...++++++++++++...+++%0A6716++++++++++2019++++++++++Local+++%0A6717++++++++++2019++++++++++Local+++%0A6718++++++++++2007++++++++++Local+++%0A6719++++++++++2014++++++++++Local+++%0A6720++++++++++2019++++++++++Local+++%0A%0A++++++++++++++++++++++++++++++++++++++++++++++Agency++%5C%0A0++++++++++++Equal+Employment+Opportunity+Commission+++%0A1+++++++++++++++++Securities+and+Exchange+Commission+++%0A2+++++Office+of+Federal+Contract+Compliance+Programs+++%0A3+++++++++++++++++++++National+Labor+Relations+Board+++%0A4+++++++++++++++++Securities+and+Exchange+Commission+++%0A...++++++++++++++++++++++++++++++++++++++++++++++...+++%0A6716++++++++++++++++Manhattan+(NY)+District

2000 SEC/FINRA Securities and Exchange Commission Dain Rauscher Incorporated NYSE: RY Dain Rauscher Incorporated agreed to pay nearly $13 million to settle allegations arising out of the SEC's investigation into the practice in municipal bond transactions known as yield burning. nan 4/6/2000 https://www.sec.gov/litigation/litreleases/lr16505.htm


ic| href: '/?sa=X&ved=0ahUKEwjdifeZlvuCAxU5vokEHTDUBBEQOwgC'
ic| href: '/search?q=Penalty+Year+++Agency+Short++%5C%0A0+++++++++++++2000++Other+Federal+++%0A1+++++++++++++2000++++++SEC/FINRA+++%0A2+++++++++++++2000++Other+Federal+++%0A3+++++++++++++2000++Other+Federal+++%0A4+++++++++++++2000++++++SEC/FINRA+++%0A...++++++++++++...++++++++++++...+++%0A6716++++++++++2019++++++++++Local+++%0A6717++++++++++2019++++++++++Local+++%0A6718++++++++++2007++++++++++Local+++%0A6719++++++++++2014++++++++++Local+++%0A6720++++++++++2019++++++++++Local+++%0A%0A++++++++++++++++++++++++++++++++++++++++++++++Agency++%5C%0A0++++++++++++Equal+Employment+Opportunity+Commission+++%0A1+++++++++++++++++Securities+and+Exchange+Commission+++%0A2+++++Office+of+Federal+Contract+Compliance+Programs+++%0A3+++++++++++++++++++++National+Labor+Relations+Board+++%0A4+++++++++++++++++Securities+and+Exchange+Commission+++%0A...++++++++++++++++++++++++++++++++++++++++++++++...+++%0A6716++++++++++++++++Manhattan+(NY)+District